## EX3

Hongru He<br>
01/21/2026

### 1. Setup and Data Loading
First, import the necessary libraries and load the datasets.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Quick inspection
print("Train shape:", train_data.shape)
train_data.head()

Train shape: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 2. Building the Preprocessing Pipeline
Handle missing values (like Age and Embarked) and convert categorical columns (like Sex and Embarked) into numbers. Scikit-Learn's Pipeline and ColumnTransformer make this cleaner.

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# pipeline for numerical attributes: impute missing values with median, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('scaler', StandardScaler())
])

# pipeline for categorical attributes: impute with most frequent, then one-hot encode
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder(sparse_output=False))
])

# Define which columns are which
num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

# Combine them
preprocess_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

# Prepare the training data
X_train = train_data.drop("Survived", axis=1)
y_train = train_data["Survived"]

X_train_prepared = preprocess_pipeline.fit_transform(X_train)

### 3. Training and Comparing Models
Now, train a Stochastic Gradient Descent (SGDClassifier) and a RandomForestClassifier and compare them using Cross-Validation.

In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 1. SGD Classifier
sgd_clf = SGDClassifier(random_state=42)
sgd_scores = cross_val_score(sgd_clf, X_train_prepared, y_train, cv=10, scoring="accuracy")

# 2. Random Forest Classifier
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, X_train_prepared, y_train, cv=10, scoring="accuracy")

print("SGD Mean Accuracy:", sgd_scores.mean())
print("Random Forest Mean Accuracy:", forest_scores.mean())

SGD Mean Accuracy: 0.7823220973782771
Random Forest Mean Accuracy: 0.8160049937578027


### 4. Fine-Tuning the Best Model
Random Forest typically performs better. We can fine-tune its hyperparameters (like n_estimators and max_features) using GridSearchCV.

In [4]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [30, 100, 200], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(forest_clf, param_grid, cv=5,
                           scoring='accuracy',
                           return_train_score=True)

grid_search.fit(X_train_prepared, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Estimator:", grid_search.best_estimator_)

Best Parameters: {'max_features': 8, 'n_estimators': 200}
Best Estimator: RandomForestClassifier(max_features=8, n_estimators=200, random_state=42)


### 5. Final Prediction on Test Set
Finally, use the best model found to predict the survival for the passengers in test.csv.

In [6]:
# Select the best model from grid search
final_model = grid_search.best_estimator_

# Preprocess the test set (NOTE: use transform(), not fit_transform()!)
X_test = test_data
X_test_prepared = preprocess_pipeline.transform(X_test)

# Make predictions
final_predictions = final_model.predict(X_test_prepared)

# Create submission DataFrame
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": final_predictions
})

# Save to CSV
submission.to_csv("submission.csv", index=False)
print("Submission file created successfully!")

Submission file created successfully!


## Summary of Findings
### 1. Submission Statistics & Sanity Check

- **Survival Rate Consistency:** The predicted survival rate in your submission is **36.60%**. This is very consistent with the actual survival rate found in the training data, which is **38.38%**. This suggests your model has learned the general distribution of the target class well and is not heavily over-predicting or under-predicting survival.

- **Class Distribution:** The predictions show that approximately **63%** of passengers are predicted to perish (0) and **37%** to survive (1).

### 2. Methodology Evaluation The workflow implemented for this task is robust and follows best practices:

- **Data Preprocessing:** You successfully utilized a ColumnTransformer and Pipeline to handle heterogeneous data. This included SimpleImputer for missing values (median for numbers, most frequent for categories), StandardScaler for numerical features, and OneHotEncoder for categorical variables.

- **Model Selection:** You compared a linear model (`SGDClassifier`) against an ensemble method (`RandomForestClassifier`). The Random Forest correctly outperformed the SGD model (Accuracy ~81% vs ~78%), which is expected as tree-based models handle non-linear relationships and interactions (like Age vs. Class) better than simple linear classifiers.

- **Hyperparameter Tuning:** You applied GridSearchCV to fine-tune the Random Forest, optimizing parameters like n_estimators and max_features. This ensures the model is not just using defaults but is tailored to the specific dataset.

### 3. Conclusion
The notebook successfully completes the objective. The transition from a simple SGD classifier to a tuned Random Forest represents a solid improvement in model complexity and performance. The final `submission.csv` appears statistically sound and ready for submission to the competition.